# Project - Week 4
## Work Plan
### 1. Systematic Study and Practice of Machine Learning

### 2. Application to Thesis: Further Dataset Optimization

### 3. Data Cleaning and Feature Engineering

### 4. Organization and Documentation with Graphics and Annotations


In [210]:
import pandas as pd
df = pd.read_excel("Data_4.xlsx")
selected_columns = [ 'EmpDepartment' ,  'EmpEducationLevel', 'EmpEnvironmentSatisfaction', 'EmpHourlyRate', 'EmpJobInvolvement',  'EmpJobLevel', 'EmpJobSatisfaction', 'OverTime', 'EmpRelationshipSatisfaction','EmpWorkLifeBalance','Attrition']
df_selected = df[selected_columns]
df_selected.to_excel('processed_data.xlsx', index=False)

### Perform column extraction on the dataset.

In [211]:
missing_values = df_selected.isnull().sum()
print(missing_values)

EmpDepartment                  0
EmpEducationLevel              0
EmpEnvironmentSatisfaction     0
EmpHourlyRate                  0
EmpJobInvolvement              0
EmpJobLevel                    0
EmpJobSatisfaction             0
OverTime                       0
EmpRelationshipSatisfaction    0
EmpWorkLifeBalance             0
Attrition                      0
dtype: int64


### Check for missing values in each column.

In [212]:
# Define the names of the competence rating columns.
comp_columns = ['EmpEducationLevel', 'EmpEnvironmentSatisfaction', 'EmpHourlyRate', 'EmpJobInvolvement',  'EmpJobLevel', 'EmpJobSatisfaction', 'OverTime', 'EmpRelationshipSatisfaction','EmpWorkLifeBalance','Attrition']

# Iterate through these columns to check for anomalies.
for column in comp_columns:
    if column in df_selected.columns: 
        outliers = df_selected[~df_selected[column].isin([1, 2, 3, 4])]
        if not outliers.empty:
            print(f"Anomalies in column '{column}':")
            print(outliers)

Anomalies in column 'EmpEducationLevel':
               EmpDepartment  EmpEducationLevel  EmpEnvironmentSatisfaction  \
12                     Sales                  5                           3   
24               Development                  5                           2   
140                    Sales                  5                           1   
198   Research & Development                  5                           4   
209   Research & Development                  5                           1   
284          Human Resources                  5                           4   
290   Research & Development                  5                           1   
392   Research & Development                  5                           1   
403   Research & Development                  5                           2   
407   Research & Development                  5                           3   
421                    Sales                  5                           1   
441   Resea

### Check for Anomalies

In [213]:
columns_to_process = ['EmpEducationLevel' , 'EmpJobLevel']
for column in columns_to_process:
    df_selected.loc[:, column] = df_selected[column].apply(lambda x: max(1, min(x, 4)))
for column in columns_to_process:
    if df_selected[column].min() < 1 or df_selected[column].max() > 4:
        print(f"Column {column} still has anomalies")
    else:
        print(f"Column {column} has no anomalies")

Column EmpEducationLevel has no anomalies
Column EmpJobLevel has no anomalies


### Normalize data with minor differences.

In [214]:
# Define the names of the competence rating columns.
comp_columns = ['EmpEducationLevel', 'EmpEnvironmentSatisfaction', 'EmpHourlyRate', 'EmpJobInvolvement',  'EmpJobLevel', 'EmpJobSatisfaction', 'OverTime', 'EmpRelationshipSatisfaction','EmpWorkLifeBalance','Attrition']

# Iterate through these columns to check for anomalies.
for column in comp_columns:
    outliers = df_selected[~df_selected[column].isin([1, 2, 3, 4])]
    if not outliers.empty:
        print(f"Anomalies in column '{column}':")
        print(outliers)

### No more Anomalies

In [215]:
# Define a mapping of original column names to new column names.
column_mapping = {
    'EmpEducationLevel': 'Education Level',
    'EmpEnvironmentSatisfaction': 'Leadership',
    'EmpHourlyRate': 'Communication Skills',
    'EmpJobInvolvement': 'Self-Motivation',
    'EmpJobLevel': 'Problem-Solving Skills',
    'EmpJobSatisfaction': 'Technical Proficiency',
    'OverTime': 'Emotional Intelligence',
    'EmpRelationshipSatisfaction': 'Creativity',
    'EmpWorkLifeBalance': 'Teamwork',
    'Attrition': 'Decision-Making'
}

# Update the column names in the comp_columns list.
comp_columns = [column_mapping[col] if col in column_mapping else col for col in comp_columns]

df_selected = df_selected.rename(columns=column_mapping)



print(df_selected.columns)



Index(['EmpDepartment', 'Education Level', 'Leadership',
       'Communication Skills', 'Self-Motivation', 'Problem-Solving Skills',
       'Technical Proficiency', 'Emotional Intelligence', 'Creativity',
       'Teamwork', 'Decision-Making'],
      dtype='object')


### Modify the existing competences to match the target competences.

In [216]:
# Replace 'Human Resources' and 'Finance' in the 'EmpDepartment' column with 'Administration'.
df_selected['EmpDepartment'].replace(['Human Resources', 'Finance'], 'Administration', inplace=True)

print(df_selected['EmpDepartment'].unique())

['Sales' 'Administration' 'Development' 'Data Science'
 'Research & Development']


In [217]:
df_selected.to_excel('cleaned_data.xlsx', index=False)


### Merge 'Human Resources' and 'Finance' departments into 'Administration'.

In [218]:
print(df_selected.dtypes)

EmpDepartment             object
Education Level            int64
Leadership                 int64
Communication Skills       int64
Self-Motivation            int64
Problem-Solving Skills     int64
Technical Proficiency      int64
Emotional Intelligence     int64
Creativity                 int64
Teamwork                   int64
Decision-Making            int64
dtype: object


# Feature Engineering
By applying different weight ratios for various departments, employees are scored accordingly. This process distinguishes two category labels: 'Suitable' and 'Not Suitable' for the position

In [219]:
weights = {
    'Administration': {
        'Leadership': 0.20, 'Communication Skills': 0.20, 'Emotional Intelligence': 0.15, 'Decision-Making': 0.15, 'Teamwork': 0.10,
        'Education Level': 0.10, 'Problem-Solving Skills': 0.05, 'Creativity': 0.05, 'Technical Proficiency': 0.05, 'Self-Motivation': 0.05
    },
    'Sales': {
        'Communication Skills': 0.25, 'Emotional Intelligence': 0.20, 'Self-Motivation': 0.20, 'Creativity': 0.10, 'Teamwork': 0.10,
        'Education Level': 0.05, 'Problem-Solving Skills': 0.05, 'Leadership': 0.05, 'Technical Proficiency': 0, 'Decision-Making': 0
    },
    'Development': {
        'Technical Proficiency': 0.25, 'Problem-Solving Skills': 0.20, 'Creativity': 0.15, 'Teamwork': 0.10, 'Decision-Making': 0.10,
        'Education Level': 0.05, 'Emotional Intelligence': 0.05, 'Leadership': 0.05, 'Communication Skills': 0.05, 'Self-Motivation': 0
    },
    'Data Science': {
        'Technical Proficiency': 0.25, 'Problem-Solving Skills': 0.20, 'Creativity': 0.15, 'Education Level': 0.15, 'Decision-Making': 0.10,
        'Teamwork': 0.05, 'Emotional Intelligence': 0.05, 'Leadership': 0.05, 'Communication Skills': 0.05, 'Self-Motivation': 0.05
    },
    'Research & Development': {
        'Technical Proficiency': 0.25, 'Problem-Solving Skills': 0.20, 'Creativity': 0.20, 'Teamwork': 0.10, 'Education Level': 0.10,
        'Decision-Making': 0.05, 'Emotional Intelligence': 0.05, 'Leadership': 0.05, 'Communication Skills': 0.05, 'Self-Motivation': 0.05
    }
}


for department, dept_weights in weights.items():
    df_selected[f'{department}_Score'] = sum(df_selected[comp] * weight for comp, weight in dept_weights.items() if comp in df_selected.columns)
    
threshold = 2  
for department in weights.keys():
    df_selected[f'Suitable_for_{department}'] = df_selected[f'{department}_Score'].apply(lambda x: 1 if x >= threshold else 0)

df_selected['Suit_Current_Department'] = df_selected.apply(lambda row: row[f'Suitable_for_{row["EmpDepartment"]}'], axis=1)




### Weight distribution, output scores for each department.‘Suit’ = 1 、‘Not Suit’ = 0. 

In [220]:
print(df_selected.dtypes)

EmpDepartment                           object
Education Level                          int64
Leadership                               int64
Communication Skills                     int64
Self-Motivation                          int64
Problem-Solving Skills                   int64
Technical Proficiency                    int64
Emotional Intelligence                   int64
Creativity                               int64
Teamwork                                 int64
Decision-Making                          int64
Administration_Score                   float64
Sales_Score                            float64
Development_Score                      float64
Data Science_Score                     float64
Research & Development_Score           float64
Suitable_for_Administration              int64
Suitable_for_Sales                       int64
Suitable_for_Development                 int64
Suitable_for_Data Science                int64
Suitable_for_Research & Development      int64
Suit_Current_

In [221]:
df_selected = pd.get_dummies(df_selected, columns=[ 'EmpDepartment'])

One-hot encoding creates new binary columns

In [222]:
print(df_selected.dtypes)


Education Level                           int64
Leadership                                int64
Communication Skills                      int64
Self-Motivation                           int64
Problem-Solving Skills                    int64
Technical Proficiency                     int64
Emotional Intelligence                    int64
Creativity                                int64
Teamwork                                  int64
Decision-Making                           int64
Administration_Score                    float64
Sales_Score                             float64
Development_Score                       float64
Data Science_Score                      float64
Research & Development_Score            float64
Suitable_for_Administration               int64
Suitable_for_Sales                        int64
Suitable_for_Development                  int64
Suitable_for_Data Science                 int64
Suitable_for_Research & Development       int64
Suit_Current_Department                 

In [223]:
columns_to_convert = [
    
    'EmpDepartment_Administration', 'EmpDepartment_Data Science',
    'EmpDepartment_Development', 'EmpDepartment_Research & Development',
    'EmpDepartment_Sales',
]
df_selected[columns_to_convert] = df_selected[columns_to_convert].astype(int)


Convert one-hot encoded columns to integer type for machine learning model compatibility.

In [224]:
print(df_selected[columns_to_convert].dtypes)


EmpDepartment_Administration            int32
EmpDepartment_Data Science              int32
EmpDepartment_Development               int32
EmpDepartment_Research & Development    int32
EmpDepartment_Sales                     int32
dtype: object


In [225]:
print(df_selected.dtypes)

Education Level                           int64
Leadership                                int64
Communication Skills                      int64
Self-Motivation                           int64
Problem-Solving Skills                    int64
Technical Proficiency                     int64
Emotional Intelligence                    int64
Creativity                                int64
Teamwork                                  int64
Decision-Making                           int64
Administration_Score                    float64
Sales_Score                             float64
Development_Score                       float64
Data Science_Score                      float64
Research & Development_Score            float64
Suitable_for_Administration               int64
Suitable_for_Sales                        int64
Suitable_for_Development                  int64
Suitable_for_Data Science                 int64
Suitable_for_Research & Development       int64
Suit_Current_Department                 

Check the types

In [226]:
# Output the modified DataFrame to an Excel file.
df_selected.to_excel('modified_data.xlsx', index=False)

### Print the prototype of the data.


In [227]:
suit_mapping = {'Suit': 1, 'NoSuit': 0}

columns_to_convert = [
    'Suitable_for_Administration', 
    'Suitable_for_Sales', 
    'Suitable_for_Development', 
    'Suitable_for_Data Science', 
    'Suitable_for_Research & Development', 
    'Suit_Current_Department'
]

for col in columns_to_convert:
    df_selected[col] = df_selected[col].replace(suit_mapping)


In [228]:
print(df_selected.dtypes)

Education Level                           int64
Leadership                                int64
Communication Skills                      int64
Self-Motivation                           int64
Problem-Solving Skills                    int64
Technical Proficiency                     int64
Emotional Intelligence                    int64
Creativity                                int64
Teamwork                                  int64
Decision-Making                           int64
Administration_Score                    float64
Sales_Score                             float64
Development_Score                       float64
Data Science_Score                      float64
Research & Development_Score            float64
Suitable_for_Administration               int64
Suitable_for_Sales                        int64
Suitable_for_Development                  int64
Suitable_for_Data Science                 int64
Suitable_for_Research & Development       int64
Suit_Current_Department                 

By encoding, a dataset consisting solely of numerical values is achieved, meeting the requirements of a machine learning dataset

In [229]:
from sklearn.model_selection import train_test_split

X = df_selected.drop('Suit_Current_Department', axis=1)  # Features
y = df_selected['Suit_Current_Department']               # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


### Split the dataset into training and test sets

In [230]:
print(X_train.dtypes)


Education Level                           int64
Leadership                                int64
Communication Skills                      int64
Self-Motivation                           int64
Problem-Solving Skills                    int64
Technical Proficiency                     int64
Emotional Intelligence                    int64
Creativity                                int64
Teamwork                                  int64
Decision-Making                           int64
Administration_Score                    float64
Sales_Score                             float64
Development_Score                       float64
Data Science_Score                      float64
Research & Development_Score            float64
Suitable_for_Administration               int64
Suitable_for_Sales                        int64
Suitable_for_Development                  int64
Suitable_for_Data Science                 int64
Suitable_for_Research & Development       int64
EmpDepartment_Administration            

### Since the "EmpNumber" column is displayed as an "object" type and does not significantly impact the data, it can therefore be removed

# Machine Learning Model Testing
## 1.Random Forest Classifier

In [231]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.9944444444444445
              precision    recall  f1-score   support

           0       0.89      1.00      0.94        17
           1       1.00      0.99      1.00       343

    accuracy                           0.99       360
   macro avg       0.95      1.00      0.97       360
weighted avg       1.00      0.99      0.99       360



In [232]:
import joblib

# 保存训练好的模型到一个文件
joblib.dump(model, 'random_forest_model.pkl')
print("模型已保存为 'random_forest_model.pkl'")


模型已保存为 'random_forest_model.pkl'


In [233]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 加载数据
file_path = 'modified_data.xlsx'
data = pd.read_excel(file_path)

# 选择特征和目标变量
X = data[['Education Level', 'Leadership', 'Communication Skills', 'Self-Motivation',
          'Problem-Solving Skills', 'Technical Proficiency', 'Emotional Intelligence',
          'Creativity', 'Teamwork', 'Decision-Making']]

# 目标变量（例如：预测是否适合 Development 部门）
y = data['Suitable_for_Development']

# 将数据拆分为训练集和测试集（80% 训练，20% 测试）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建并训练随机森林分类器
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 预测测试集标签
y_pred = model.predict(X_test)

# 输出模型的评估指标
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9583333333333334
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.56      0.64        16
           1       0.97      0.99      0.98       224

    accuracy                           0.96       240
   macro avg       0.86      0.77      0.81       240
weighted avg       0.95      0.96      0.96       240



In [234]:
import joblib

# 保存训练好的模型到一个文件
joblib.dump(model, 'random_forest_model2.pkl')
print("模型已保存为 'random_forest_model2.pkl'")

模型已保存为 'random_forest_model2.pkl'


### This model performs exceptionally well in predicting category 1, with both high precision and recall.
### The prediction for category 0 is also quite accurate, though slightly lower compared to category 1.
### The overall accuracy is very high, indicating that the model makes correct predictions for the majority of samples.

## 2.Support Vector Machine, SVM

In [170]:
from sklearn.svm import SVC

model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.9638888888888889
              precision    recall  f1-score   support

           0       0.87      0.75      0.81        36
           1       0.97      0.99      0.98       324

    accuracy                           0.96       360
   macro avg       0.92      0.87      0.89       360
weighted avg       0.96      0.96      0.96       360



#### for this dataset, using SVM might encounter issues such as data imbalance or parameter mismatch

## 3.Decision Tree

In [171]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.9722222222222222
              precision    recall  f1-score   support

           0       0.88      0.83      0.86        36
           1       0.98      0.99      0.98       324

    accuracy                           0.97       360
   macro avg       0.93      0.91      0.92       360
weighted avg       0.97      0.97      0.97       360



## 4.Logistic Regression

In [172]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.9472222222222222
              precision    recall  f1-score   support

           0       0.73      0.75      0.74        36
           1       0.97      0.97      0.97       324

    accuracy                           0.95       360
   macro avg       0.85      0.86      0.86       360
weighted avg       0.95      0.95      0.95       360



D:\Software\Anacoda\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### It is a ConvergenceWarning indicating that the logistic regression model did not converge during training

## 5.K-Nearest Neighbors, KNN

In [173]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.9583333333333334
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        36
           1       0.96      0.99      0.98       324

    accuracy                           0.96       360
   macro avg       0.93      0.83      0.87       360
weighted avg       0.96      0.96      0.96       360



#### The data type is not suitable for KNN

## conclusion
### Model Testing and Evaluation:

I have tested several popular machine learning models, including Support Vector Machine (SVM), Logistic Regression, Decision Tree, K-Nearest Neighbors (KNN), and Random Forest Classifier (RFC).
During the testing process, I found that models like SVM, Logistic Regression, and KNN encountered errors or warnings in some cases, which might be due to data imbalance, mismatch in model parameters, or the nature of the algorithms themselves.
### Performance of Random Forest Classifier and Decision Tree:

The Random Forest Classifier (RFC) and Decision Tree (DT) performed well in my tests, with no significant errors or issues.
The accuracy of RFC was about 98.61%, while DT achieved an accuracy of 97.5%, both demonstrating high predictive accuracy.
### Future Research Direction:

Based on the current test results and the performance of the models, I plan to continue using Random Forest Classifier and Decision Tree in my future research.
Both models not only offer high accuracy but are also relatively easy to handle with large datasets and have lower requirements for data preprocessing.
### Next Steps:

In future work, I might further fine-tune and optimize the parameters of these two models to enhance their performance.
Additionally, exploring more techniques in feature engineering and model tuning could help improve the models' accuracy and interpretability.


## 1
Define Department Feature Weights
First, define the feature weights for each department.

Department A values Leadership and Communication Skills more.
Department B values Technical Proficiency and Problem-Solving Skills more.
Department C values Customer Service and Teamwork more.
Department D values Creativity and Innovation more.
Department E values Attention to Detail and Execution more.

In [174]:
import pandas as pd

# Load data
df_selected = pd.read_excel('modified_data.xlsx')

# Define feature weights for each department
weights = {
    'Administration': {'Leadership': 0.25, 'Communication Skills': 0.20, 'Emotional Intelligence': 0.20, 'Decision-Making': 0.15, 'Teamwork': 0.10, 'Education Level': 0.05, 'Problem-Solving Skills': 0.05, 'Creativity': 0, 'Technical Proficiency': 0, 'Self-Motivation': 0},
    'Sales': {'Communication Skills': 0.25, 'Emotional Intelligence': 0.20, 'Self-Motivation': 0.20, 'Creativity': 0.15, 'Teamwork': 0.10, 'Education Level': 0.05, 'Problem-Solving Skills': 0.05, 'Leadership': 0, 'Technical Proficiency': 0, 'Decision-Making': 0},
    'Development': {'Technical Proficiency': 0.25, 'Problem-Solving Skills': 0.20, 'Creativity': 0.15, 'Teamwork': 0.10, 'Decision-Making': 0.15, 'Education Level': 0.05, 'Emotional Intelligence': 0.05, 'Leadership': 0, 'Communication Skills': 0, 'Self-Motivation': 0.05},
    'Data Science': {'Technical Proficiency': 0.25, 'Problem-Solving Skills': 0.20, 'Creativity': 0.20, 'Education Level': 0.15, 'Decision-Making': 0.10, 'Teamwork': 0.05, 'Emotional Intelligence': 0.05, 'Leadership': 0, 'Communication Skills': 0, 'Self-Motivation': 0},
    'Research & Development': {'Technical Proficiency': 0.25, 'Problem-Solving Skills': 0.20, 'Creativity': 0.20, 'Teamwork': 0.15, 'Education Level': 0.10, 'Decision-Making': 0.05, 'Emotional Intelligence': 0.05, 'Leadership': 0, 'Communication Skills': 0, 'Self-Motivation': 0}
}


### 2.Calculate suitability scores for each department

In [175]:
for department, dept_weights in weights.items():
    df_selected[f'{department}_Score'] = df_selected.apply(lambda row: sum(row[feature] * weight for feature, weight in dept_weights.items()), axis=1)

### 3.Determine the most suitable department for each employee

In [176]:
df_selected['recommended_department'] = df_selected.apply(lambda row: max(weights.keys(), key=lambda dept: row[f'{dept}_Score']), axis=1)

### 4.Print scores and recommended department for the first 10 employees

In [178]:
columns_to_display = [ 'Education Level', 'Leadership', 'Communication Skills', 'Self-Motivation', 'Problem-Solving Skills', 'Technical Proficiency', 'Emotional Intelligence', 'Creativity', 'Teamwork', 'Decision-Making'] + [f'{department}_Score' for department in weights.keys()] + ['recommended_department']
print(df_selected[columns_to_display].head(10))

   Education Level  Leadership  Communication Skills  Self-Motivation  \
0                3           4                     3                3   
1                4           4                     2                3   
2                4           4                     2                2   
3                4           2                     3                2   
4                4           1                     4                3   
5                2           4                     2                3   
6                4           4                     3                3   
7                2           1                     3                1   
8                3           1                     3                4   
9                3           3                     4                3   

   Problem-Solving Skills  Technical Proficiency  Emotional Intelligence  \
0                       2                      4                       1   
1                       2                   

In [179]:
import pandas as pd
import numpy as np

# Assume that your data frame df_selected contains the ratings and characteristics data of each employee
#Create a sample data frame
data = {
    'employee_id': [1, 2, 3],
    'score': [75, 80, 85],
    'Leadership': [3, 4, 5],
    'Communication Skills': [4, 3, 4],
    'Self-Motivation': [5, 4, 3],
    'Problem-Solving Skills': [3, 4, 5],
    'Technical Proficiency': [4, 3, 4],
    'Emotional Intelligence': [5, 4, 3],
    'Creativity': [3, 4, 5],
    'Teamwork': [4, 3, 4],
    'Decision-Making': [5, 4, 3]
}

df_selected = pd.DataFrame(data)

# Generate 12 months dates
months = pd.date_range(start='2023-01-01', periods=12, freq='M')

expanded_data = []
for _, row in df_selected.iterrows():
    for month in months:
        expanded_row = row.copy()
        expanded_row['date'] = month
        expanded_data.append(expanded_row)

df_monthly = pd.DataFrame(expanded_data)

np.random.seed(42)
df_monthly['score'] = df_monthly['score'].apply(lambda x: x + np.random.randint(-5, 6))

#Print the generated monthly data
print(df_monthly.head(24))  

   employee_id  score  Leadership  Communication Skills  Self-Motivation  \
0            1     76           3                     4                5   
0            1     73           3                     4                5   
0            1     80           3                     4                5   
0            1     77           3                     4                5   
0            1     74           3                     4                5   
0            1     76           3                     4                5   
0            1     79           3                     4                5   
0            1     72           3                     4                5   
0            1     76           3                     4                5   
0            1     80           3                     4                5   
0            1     80           3                     4                5   
0            1     77           3                     4                5   
1           

In [180]:
# 将df_monthly保存为CSV文件，文件名可以根据需要修改
df_monthly.to_csv('output_data.csv', index=False)


In [181]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
import numpy as np

# Step 3: Normalize data and prepare training data
employee_id = 1
employee_data = df_monthly[df_monthly['employee_id'] == employee_id]['score'].values

# Ensure there is enough data for training
if len(employee_data) < 12:
    raise ValueError("Not enough data to perform time series prediction")

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
employee_data = scaler.fit_transform(employee_data.reshape(-1, 1))

X_train, y_train = [], []
n_future = 1
n_past = 11

# Generate training data
for i in range(n_past, len(employee_data) - n_future + 1):
    X_train.append(employee_data[i - n_past:i, 0])
    y_train.append(employee_data[i + n_future - 1:i + n_future, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Debugging: Print shapes and contents before reshaping
print("Contents of X_train:", X_train)
print("Shape of X_train before reshaping:", X_train.shape)
print("Contents of y_train:", y_train)
print("Shape of y_train:", y_train.shape)

# Ensure X_train has the correct shape
if len(X_train.shape) == 2:
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
else:
    raise ValueError(f"X_train does not have the expected shape for reshaping. Shape found: {X_train.shape}")

# Debugging: Print shapes after reshaping
print("Shape of X_train after reshaping:", X_train.shape)

# Step 4: Build and train the LSTM model
model = Sequential()

# Use Input layer to define input shape
model.add(Input(shape=(X_train.shape[1], 1)))

# Add LSTM layers
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))

# Add output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=2)

# Step 5: Prepare test data and make predictions
test_data = employee_data[-n_past:].reshape(1, n_past, 1)

# Predict the score for the next month
predicted_score = model.predict(test_data)
predicted_score = scaler.inverse_transform(predicted_score)

# Step 6: Print the predicted score for the next month
print(f'Predicted score for the next month: {predicted_score[0][0]:.2f}')



Contents of X_train: [[0.5   0.125 1.    0.625 0.25  0.5   0.875 0.    0.5   1.    1.   ]]
Shape of X_train before reshaping: (1, 11)
Contents of y_train: [[0.625]]
Shape of y_train: (1, 1)
Shape of X_train after reshaping: (1, 11, 1)
Epoch 1/20
1/1 - 1s - 1s/step - loss: 0.3832
Epoch 2/20
1/1 - 0s - 16ms/step - loss: 0.3237
Epoch 3/20
1/1 - 0s - 16ms/step - loss: 0.2693
Epoch 4/20
1/1 - 0s - 16ms/step - loss: 0.2192
Epoch 5/20
1/1 - 0s - 16ms/step - loss: 0.1728
Epoch 6/20
1/1 - 0s - 16ms/step - loss: 0.1299
Epoch 7/20
1/1 - 0s - 16ms/step - loss: 0.0910
Epoch 8/20
1/1 - 0s - 17ms/step - loss: 0.0568
Epoch 9/20
1/1 - 0s - 15ms/step - loss: 0.0287
Epoch 10/20
1/1 - 0s - 16ms/step - loss: 0.0090
Epoch 11/20
1/1 - 0s - 16ms/step - loss: 2.5007e-04
Epoch 12/20
1/1 - 0s - 16ms/step - loss: 0.0041
Epoch 13/20
1/1 - 0s - 15ms/step - loss: 0.0177
Epoch 14/20
1/1 - 0s - 16ms/step - loss: 0.0314
Epoch 15/20
1/1 - 0s - 16ms/step - loss: 0.0369
Epoch 16/20
1/1 - 0s - 12ms/step - loss: 0.0335
Epoc

In [89]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense



# Step 3: Normalize data and prepare training data
employee_id = 1
employee_data = df_monthly[df_monthly['employee_id'] == employee_id]['score'].values

# Ensure there is enough data for training
if len(employee_data) < 12:
    raise ValueError("Not enough data to perform time series prediction")

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
employee_data = scaler.fit_transform(employee_data.reshape(-1, 1))

X_train, y_train = [], []
n_future = 1
n_past = 11

# Generate training data
for i in range(n_past, len(employee_data) - n_future + 1):
    X_train.append(employee_data[i - n_past:i, 0])
    y_train.append(employee_data[i + n_future - 1:i + n_future, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Debugging: Print shapes and contents before reshaping
print("Contents of X_train:", X_train)
print("Shape of X_train before reshaping:", X_train.shape)
print("Contents of y_train:", y_train)
print("Shape of y_train:", y_train.shape)

# Ensure X_train has the correct shape
if len(X_train.shape) == 2:
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
else:
    raise ValueError(f"X_train does not have the expected shape for reshaping. Shape found: {X_train.shape}")

# Debugging: Print shapes after reshaping
print("Shape of X_train after reshaping:", X_train.shape)

# Step 4: Build and train the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=2)

# Step 5: Prepare test data and make predictions
test_data = employee_data[-n_past:].reshape(1, n_past, 1)

predicted_score = model.predict(test_data)
predicted_score = scaler.inverse_transform(predicted_score)

# Step 6: Print the predicted score for the next month
print(f'Predicted score for the next month: {predicted_score[0][0]:.2f}')

Contents of X_train: [[0.5   0.125 1.    0.625 0.25  0.5   0.875 0.    0.5   1.    1.   ]]
Shape of X_train before reshaping: (1, 11)
Contents of y_train: [[0.625]]
Shape of y_train: (1, 1)
Shape of X_train after reshaping: (1, 11, 1)
Epoch 1/20


D:\Software\Anacoda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 - 2s - 2s/step - loss: 0.4688
Epoch 2/20
1/1 - 0s - 16ms/step - loss: 0.4085
Epoch 3/20
1/1 - 0s - 16ms/step - loss: 0.3543
Epoch 4/20
1/1 - 0s - 16ms/step - loss: 0.3049
Epoch 5/20
1/1 - 0s - 16ms/step - loss: 0.2594
Epoch 6/20
1/1 - 0s - 16ms/step - loss: 0.2168
Epoch 7/20
1/1 - 0s - 16ms/step - loss: 0.1769
Epoch 8/20
1/1 - 0s - 16ms/step - loss: 0.1393
Epoch 9/20
1/1 - 0s - 18ms/step - loss: 0.1042
Epoch 10/20
1/1 - 0s - 12ms/step - loss: 0.0720
Epoch 11/20
1/1 - 0s - 16ms/step - loss: 0.0436
Epoch 12/20
1/1 - 0s - 20ms/step - loss: 0.0207
Epoch 13/20
1/1 - 0s - 16ms/step - loss: 0.0053
Epoch 14/20
1/1 - 0s - 16ms/step - loss: 7.0920e-06
Epoch 15/20
1/1 - 0s - 16ms/step - loss: 0.0060
Epoch 16/20
1/1 - 0s - 16ms/step - loss: 0.0199
Epoch 17/20
1/1 - 0s - 16ms/step - loss: 0.0323
Epoch 18/20
1/1 - 0s - 16ms/step - loss: 0.0364
Epoch 19/20
1/1 - 0s - 16ms/step - loss: 0.0323
Epoch 20/20
1/1 - 0s - 16ms/step - loss: 0.0235
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
Predicted score for

In [124]:
# Debugging: Check the length of employee_data
print(f"Length of employee_data: {len(employee_data)}")

X_train, y_train = [], []
n_future = 1
n_past = 11

# Generate training data
if len(employee_data) < (n_past + n_future):
    raise ValueError("Not enough data to generate training samples")

for i in range(n_past, len(employee_data) - n_future + 1):
    X_train.append(employee_data[i - n_past:i, 0])
    y_train.append(employee_data[i + n_future - 1:i + n_future, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Check the shape after generation
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")


Length of employee_data: 12
Shape of X_train: (1, 11)
Shape of y_train: (1, 1)
